In [ ]:
import pandas as pd
import re
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

In [ ]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

train['data_type'] = "train"
test['data_type'] = "test"
test['Survived'] = "test"

data = pd.concat([train, test], axis=0)
data = data.drop(['Cabin'], axis=1)
data = data.dropna(how="any")

In [ ]:
def deriveTitles(name: str) -> str:
    """ name に含まれるタイトルをそれぞれのカテゴリに変換
    Args:
        name (str): 登場者の名前
    Returns:
        名前から抽出されたタイトルに従って変換されたカテゴリ (int)
    """
    title = re.search('(?:\S )(?P<title>\w*)', name).group('title')
    
    if title == "Mr":             return "adult"
    elif title == "Don":          return "gentry"
    elif title == "Dona":         return "gentry"
    elif title == "Miss":         return "miss"
    elif title == "Col":          return "military"
    elif title == "Rev":          return "other"
    elif title == "Lady":         return "gentry"
    elif title == "Master":       return "child"
    elif title == "Mme":          return "adult"
    elif title == "Captain":      return "military"
    elif title == "Dr":           return "other"
    elif title == "Mrs":          return "adult"
    elif title == "Sir":          return "gentry"
    elif title == "Jonkheer":     return "gentry"
    elif title == "Mlle":         return "miss"
    elif title == "Major":        return "military"
    elif title == "Ms":           return "miss"
    elif title == "the Countess": return "gentry"
    else:                         return "other"
    
data["title"] = data.Name.apply(deriveTitles)
# and encode these new titles for later
le = preprocessing.LabelEncoder()
titles = ['adult', 'gentry', 'miss', 'military', 'other', 'child']
le.fit(titles)
data['encodedTitle'] = le.transform(data['title']).astype('int')

In [ ]:
data = data.assign(SibSpGroup1=train['SibSp'] < 2)
data = data.assign(SibSpGroup2=train['SibSp'].between(2, 3, inclusive=True))
data = data.assign(SibSpGroup3=train['SibSp'] > 2)

In [ ]:
data = data.assign(ParChGT2=train['Parch'] > 2)
data = data.assign(familySize=data['Parch'] + data['SibSp'])

In [ ]:
def deriveChildren(age: int, parch: int) -> int:
    """ 連れている両親の数だけを抽出。18歳以上は両親がいないと仮定。
    Args:
        age (int): 搭乗者の年齢
        parch (int): 両親、子供の数
    Returns:
        両親の数 (int)
    """
    if(age < 18):
        return parch
    else:
        return 0
    
data = data.assign(children=data.apply(lambda row: deriveChildren(row['Age'], row['Parch']), axis = 1))

In [ ]:
def deriveParents(age: int, parch: int) -> int:
    """ 連れている子供の数だけを抽出。18歳未満は子供がいないと仮定。
    Args:
        age (int): 搭乗者の年齢
        parch (int): 両親、子供の数
    Returns:
        子供の数 (int)
    """
    if(age > 17):
        return parch
    else:
        return 0
    
data['parents'] = data.apply(lambda row: deriveParents(row['Age'], row['Parch']), axis = 1)

In [ ]:
def deriveResponsibleFor(children: int, SibSp: int) -> float:
    """ 自分を含めた同行者における18歳以上のの大人と責任を持つべき子供の割合を算出。
    Args:
        children (int): 子供の数
        SibSp (int): 兄弟、配偶者の数
    Returns:
        責任 (XXX)
    """
    if(children > 0):
        return children / (SibSp + 1)  # float何にする?
    else:
        return 0

data['responsibleFor'] = data.apply(lambda row: deriveResponsibleFor(row['children'], row['SibSp']), axis = 1)

In [ ]:
def deriveAccompaniedBy(parents, SibSp):
    """ 親が1以上いる場合に、自分を含めた同行者における、両親の割合を算出。
    Args:
        parents (int): 親の数
        SibSp (int): 兄弟、配偶者の数
    Returns:
        親に対する子供の割合 (XXX)
    """
    if(parents > 0): return parents / (SibSp + 1)
    else: return 0

data['accompaniedBy'] = data.apply(lambda row: deriveAccompaniedBy(row['parents'], row['SibSp']), axis = 1)

In [ ]:
def unaccompaniedChild(age: int, parch: int) -> bool:
    """ 同行者がいない16歳以下の搭乗者かどうかを判定。
    Args:
        age (int): 搭乗者の年齢
        parch (int): 同行者の数
    Returns:
        同行者がいない場合 True, いる場合 False (bool)
    """
    if((age < 16) & (parch == 0)):
        return 1
    else:
        return 0
    
data['unaccompaniedChild'] = data.apply(lambda row: unaccompaniedChild(row['Age'], row['Parch']), axis = 1)

In [ ]:
data = data.astype({"Pclass": int, "Age": int, "SibSp": int, "Parch": int})

# label encode string features
categorical_names = {}
categorical_features = ['Embarked', 'Sex', 'SibSpGroup1', 'SibSpGroup2', 'SibSpGroup3', 'ParChGT2']

for feature in categorical_features:
    le = preprocessing.LabelEncoder()
    le.fit(data[feature])
    data[feature] = le.transform(data[feature])
    categorical_names[feature] = le.classes_


data['class'] = data['Pclass'].astype(int, copy=False)
data = data.drop(['Pclass'], axis=1)
data = data.drop(['Name', 'Parch', 'SibSp', 'title', 'Ticket', 'PassengerId'], axis=1)

In [ ]:
train = data[data['data_type'] == 'train']
test = data[data['data_type'] == 'test']
X_train, y_train = train.drop(['Survived', 'data_type'], axis=1), train['Survived']
y_train = y_train.astype({"Survived": int})

test = test.drop(['Survived', 'data_type'], axis=1)

In [ ]:
X_train.to_csv('./data/preprocessed/X_train.csv', header=True, index=False)
y_train.to_csv('./data/preprocessed/y_train.csv', header=True, index=False)
test.to_csv('./data/preprocessed/test.csv', header=True, index=False)

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV

rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

param_grid =  {
    "criterion": ["gini", "entropy"],
    "max_depth":[i for i in range(1, 5)]
              }

stratifiedkfold = StratifiedKFold(n_splits=3)
grid_search = GridSearchCV(rf, param_grid, cv=stratifiedkfold, refit=True, scoring='f1')
grid_search.fit(X_train, y_train)

print("score: {}".format(grid_search.best_score_))

In [ ]:
prediction = grid_search.predict(test)
pd.Series(prediction, name='Survived').to_csv('./data/prediction.csv', header=True, index=False)